# Imports

In [24]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd


# Load Data

In [ ]:
# Load the data
input_file_foras = list(Path(os.path.join("data")).glob("PTSS_Data_Foras.xlsx"))[0]
input_file_synergy = list(Path(os.path.join("data")).glob("PTSS_Data_Synergy.xlsx"))[0]
fulltext_foras = list(Path(os.path.join("data")).glob("PTSS_Data_Foras_Fulltext.xlsx"))[
    0
]
fulltext_foras_2nd = list(
    Path(os.path.join("data")).glob("PTSS_Data_Foras_Fulltext_2ndscreener.xlsx")
)[0]
fulltext_synergy = list(
    Path(os.path.join("data")).glob("PTSS_Data_Synergy_Fulltext.xlsx")
)[0]

# Print the file names
print(
    "Results based on file: ",
    input_file_foras,
    input_file_synergy,
    fulltext_foras,
    fulltext_foras_2nd,
    fulltext_synergy,
)

# Read the foras file and filter out the duplicates
foras_unfiltered = pd.read_excel(input_file_foras)
foras_filtered = foras_unfiltered[foras_unfiltered["filter_duplicate"] != 1]

# Read the other files
synergy = pd.read_excel(input_file_synergy)
fulltext_foras = pd.read_excel(fulltext_foras)
fulltext_foras_2nd = pd.read_excel(fulltext_foras_2nd)
fulltext_synergy = pd.read_excel(fulltext_synergy)

# Print the number of rows in each file
print("Number of records in original FORAS file: ", foras_unfiltered.shape[0])
print(
    "Number of records in FORAS after filtering duplicates: ", foras_filtered.shape[0]
)
print("Number of records in SYNERGY", synergy.shape[0])
print("Number of records in FORAS fulltext", fulltext_foras.shape[0])
print("Number of records in FORAS fulltext 2nd screener", fulltext_foras_2nd.shape[0])
print("Number of records in SYNERGY fulltext", fulltext_synergy.shape[0])


# Duplicates

In [ ]:
# calculate the number of duplicates in the foras unfiltred file using duplicate_record_identifier
duplicates = foras_unfiltered[
    foras_unfiltered["duplicate_record_identifier"].notnull()
].shape[0]
print(
    "Number of duplicates in FORAS file, which will be ignored in the remainder of the analyses: ",
    duplicates,
)


# Number of PIDs, Titles and Abstracts

In [ ]:
# Function to calculate missing data and plot for a given dataset
def analyze_missing_data(dataset, dataset_name):
    # Define columns to check for missing values
    columns_to_check = ["doi", "openalex_id", "title", "abstract"]
    missing_data_counts = {col: dataset[col].isnull().sum() for col in columns_to_check}
    missing_data_counts["Both Missing"] = (
        dataset["doi"].isnull().sum() & dataset["openalex_id"].isnull().sum()
    )

    # Print the number of records without certain values
    print(f"Missing Data Analysis for {dataset_name}:")
    for category, count in missing_data_counts.items():
        print(f"  - Number of records without {category}: {count}")

    # Calculate percentages for visualization
    total_records = dataset.shape[0]
    percentages = {
        key: (value / total_records) * 100 for key, value in missing_data_counts.items()
    }

    # Plotting
    plt.figure(figsize=(10, 6))
    categories = list(percentages.keys())
    values = list(percentages.values())
    bars = plt.bar(categories, values, color=plt.cm.tab10(range(len(categories))))

    # Add title and labels
    plt.title(f"Percentage of Missing Data in Each Category ({dataset_name})")
    plt.xlabel("Missing Data Category")
    plt.ylabel("Percentage of Total Records")

    # Annotate bars
    for bar, category in zip(bars, categories):
        height = bar.get_height()
        count = missing_data_counts[category]
        plt.text(
            bar.get_x() + bar.get_width() / 2.0,
            height,
            f"{count} ({height:.1f}%)",
            ha="center",
            va="bottom",
        )

    # Display the chart
    plt.show()

    # Return results for further programmatic use
    return missing_data_counts


# Call the function for both datasets
foras_results = analyze_missing_data(foras_filtered, "Foras")
synergy_results = analyze_missing_data(synergy, "Synergy")


# Frequencies for search results

In [ ]:
def generate_frequency_overview(dataset, variable_list):
    print("Frequency Overview:\n")
    for variable in variable_list:
        if variable not in dataset.columns:
            print(f"Variable '{variable}' not found in the dataset. Skipping...\n")
            continue

        print(f"Variable: {variable}")
        value_counts = dataset[variable].value_counts(dropna=False)
        total_records = value_counts.sum()
        percentages = (value_counts / total_records * 100).round(2)
        frequency_table = pd.DataFrame(
            {"Count": value_counts, "Percentage": percentages}
        )

        print(frequency_table)
        print("\n" + "-" * 50 + "\n")


# New list of variables to analyze
variable_list_additional = [
    "search_replication",
    "search_comprehensive",
    "search_snowballing",
    "search_fulltext",
    "openalex_rank_inclusion_criteria",
    "search_openalex_inlusion_criteria",
    "search_openalex_inlusion_criteria_long",
    "openalex_rank_logistic",
    "search_openalex_logistic",
    "search_openalex_logistic_long",
    "openalex_rank_all_abstracts",
    "openalex_rank_all_abstracts_long",
    "batch",
]

generate_frequency_overview(foras_filtered, variable_list_additional)


# Frequencies for labeling decissions

In [ ]:
# List of variables to analyze
variable_list = [
    "title_eligible_Bruno",
    "TI-AB_IC1_Bruno",
    "TI-AB_IC2_Bruno",
    "TI-AB_IC3_Bruno",
    "TI-AB_IC4_Bruno",
    "TI-AB_final_label_Bruno",
    "title_eligible_Rutger",
    "TI_final_label",
    "TI-AB_final_label_Rutger",
    "TI-AB_disagreement_human-human",
    "TI-AB_IC1_LLM",
    "TI-AB_IC2_LLM",
    "TI-AB_IC3_LLM",
    "TI-AB_IC4_LLM",
    "TI-AB_final_label_LLM",
    "LLM_re-assessed",
    "TI-AB_disagreement_human-LLM",
    "TI-AB_IC1_joint",
    "TI-AB_IC2_joint",
    "TI-AB_IC3_joint",
    "TI-AB_IC4_joint",
    "TI-AB_IC1_final",
    "TI-AB_IC2_final",
    "TI-AB_IC3_final",
    "TI-AB_IC4_final",
    "TI-AB_final_label",
    "full_text_available",
    "FT_IC1_Bruno",
    "FT_IC2_Bruno",
    "FT_IC3_Bruno",
    "FT_IC4_Bruno",
    "FT_exlusion_reason_Bruno",
    "FT_inclusion_Bruno",
    "FT_inclusion_Rutger",
    "FT_exlusion_reason_Rutger",
    "FT_disagreements_Bruno-Rutger",
    "FT_IC1_joint",
    "FT_IC2_joint",
    "FT_IC3_joint",
    "FT_IC4_joint",
    "FT_IC1_final",
    "FT_IC2_final",
    "FT_IC3_final",
    "FT_IC4_final",
    "FT_final_label",
    "label_included_TIAB",
    "label_included_FT",
]

# Call the function to generate the frequency overview
generate_frequency_overview(foras_filtered, variable_list)


# Labeling combinations

In [ ]:
def analyze_inclusions(dataset, column_label, col1, col2):
    # Total count and percentages for title inclusions
    print(f"Inclusion Analysis for {column_label}:\n")
    value_counts = dataset[column_label].value_counts(dropna=False)
    total_records = value_counts.sum()
    percentages = (value_counts / total_records * 100).round(2)

    print("Counts:")
    print(value_counts)
    print("\nPercentages:")
    print(percentages)
    print(f"\nGrand Total: {total_records}\n")

    # Cross-tabulation
    print(f"Cross-tabulation of {col1} vs {col2}:\n")
    crosstab = pd.crosstab(dataset[col1], dataset[col2], margins=True, dropna=True)
    print(crosstab, "\n")

    # Add percentages to the crosstab
    print("Cross-tabulation with Percentages:\n")
    crosstab_percentages = crosstab.div(crosstab.loc["All"], axis=1).round(2) * 100
    print(crosstab_percentages, "\n")


# Call the function for analysis
analyze_inclusions(
    foras_filtered, "TI_final_label", "title_eligible_Rutger", "title_eligible_Bruno"
)


In [ ]:
# Call the function for analysis
analyze_inclusions(
    foras_filtered,
    "TI-AB_final_label",
    "TI-AB_final_label_Rutger",
    "TI-AB_final_label_Bruno",
)


In [ ]:
# Call the function for analysis
analyze_inclusions(
    foras_filtered, "FT_final_label", "FT_inclusion_Rutger", "FT_inclusion_Bruno"
)
